In [54]:
import matplotlib.pyplot as plt
import os
#from __future__ import print_function
from ipywidgets import interact#, interactive, fixed, interact_manual
import ipywidgets as widgets

In [55]:
def heatThisLog(h5File):
    wf  = qp.retrieve_hdf5_data(h5File,'WF')
    fig = plt.figure(figsize=(8, 16), dpi= 80, facecolor='w', edgecolor='k')
    plt.ylabel('Gamma')
    plt.xlabel('Theta')
    #plt.imshow(qp.abs2(wf), vmax = 1e-30)
    plt.imshow(np.log(qp.abs2(wf)))
    
def heatThis(h5File):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    wf   = qp.retrieve_hdf5_data(h5File,'WF')
    time = qp.retrieve_hdf5_data(h5File,'Time')[0]
    fig = plt.figure(figsize=(16, 16), dpi= 80, facecolor='w', edgecolor='k')
    plt.title('Time = {:10.5f} fs'.format(time))
    plt.ylabel('Gamma')
    plt.xlabel('Theta')

    #plt.imshow(qp.abs2(wf), vmax = 1e-30)
    #plt.imshow(qp.abs2(wf), cmap='PuBu_r')
    
    # this is to get a nice colorbar on the side
    ax = plt.gca()
    im = ax.imshow(qp.abs2(wf), cmap='PuBu_r')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(im, cax=cax)
    
def f(x):
    return heatThis(os.path.join(a,filesList[x]))

In [56]:
subfolders = sorted([ dir for dir in os.listdir('.') if os.path.isdir(dir) and dir != '.ipynb_checkpoints'])
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> input2_0000
1 -> input2_0001
2 -> input2_0002
3 -> input2_0003
4 -> input2_0004
5 -> input2_0005
6 -> input2_0006
7 -> input2_0007



In [57]:
a=subfolders[-1]
a

'input2_0007'

In [58]:
filesList = sorted(os.listdir(a))

In [59]:
interact(f, x = widgets.IntSlider(min=0,max=(len(filesList)-1),step=1,value=0))

interactive(children=(IntSlider(value=0, description='x', max=1224), Output()), _dom_classes=('widget-interact…

<function __main__.f(x)>

In [60]:
gamL,theL = (qp.retrieve_hdf5_data(os.path.join(a,filesList[0]),'WF')).shape

In [69]:
def sliceThetas(h5File, gamma):
    wf  = qp.retrieve_hdf5_data(h5File,'WF')
    fig = plt.figure(figsize=(10, 5), dpi= 80, facecolor='w', edgecolor='k')
    ys = wf[gamma,:]
    time = qp.retrieve_hdf5_data(h5File,'Time')[0]
    plt.title('Time = {:10.5f} fs --- gammaL = {}'.format(time,gamma))
    plt.ylim(-0.5,0.5)
    plt.plot(np.arange(ys.size), np.real(ys), linewidth=3.0)
    plt.plot(np.arange(ys.size), np.imag(ys), linewidth=3.0)

    
    
def f2(file_number,gamma):
    return sliceThetas(os.path.join(a, filesList[file_number]), gamma)

In [70]:
interact(f2, file_number = widgets.IntSlider(min=0,max=(len(filesList)-1),step=1,value=0), gamma = widgets.IntSlider(min=0,max=gamL-1,step=1,value=0))

interactive(children=(IntSlider(value=0, description='file_number', max=1224), IntSlider(value=0, description=…

<function __main__.f2(file_number, gamma)>